In [2]:
// SETUP
use std::fs::File;
use std::collections::{HashMap, HashSet};
use std::str::FromStr;
use std::io::{BufRead, BufReader, Error, ErrorKind, Read};
use std::time::Instant;

fn read_lines(path: &str) -> Result<Vec<String>, Error> {
    let file = File::open(path)?; 
    BufReader::new(file).lines().collect()
}

fn read_integers(path: &str) -> Result<Vec<i64>, Error> {
    let mut v = Vec::new();
    for line in read_lines(path)? {
        let n = line   
            .trim() 
            .parse() 
            .map_err(|e| Error::new(ErrorKind::InvalidData, e))?; 
        v.push(n);
    }
    Ok(v)
}

const DATA_ROOT: &str = "../data/";

fn data_file(problem_id: u32) -> String {
    format!("{}/{:02}.txt", DATA_ROOT, problem_id)
}

fn test_file(problem_id: u32, test_id: u32) -> String {
    format!("{}/{:02}t{}.txt", DATA_ROOT, problem_id, test_id)
}

fn extra_file(problem_id: u32, suffix: &str) -> String {
    format!("{}/{:02}.{}.txt", DATA_ROOT, problem_id, suffix)
}

In [3]:
extern crate itertools;
use itertools::Itertools;

fn problem01() {    
    let nums: Vec<i64> = read_integers(&data_file(1)).unwrap();
    let res1: i64 = nums.iter().tuple_windows().map(|(a, b)| (b > a) as i64).sum();
    let res2: i64 = nums.iter().tuple_windows::<(_, _, _)>().map(|(a, b, c)| a + b + c)
        .tuple_windows().map(|(a, b)| (b > a) as i64).sum();
    println!("Answer 1: {}\nAnswer 2: {}\n", res1, res2);
}

problem01();

Answer 1: 1557
Answer 2: 1608



In [4]:
#[derive(Debug, PartialEq, Copy, Clone)]
enum Command {
    Forward(i32),
    Up(i32),
    Down(i32),
}

impl FromStr for Command {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let mut it = s.split(' ');
        let opstr: &str = it.next().ok_or(())?;
        let val: i32 = it.next().ok_or(())?.parse::<i32>().map_err(|_|())?;
        match opstr {
            "forward" => Ok(Command::Forward(val)),
            "up" => Ok(Command::Up(val)),
            "down" => Ok(Command::Down(val)),
            _ => Err(()),
        }
    }
}

fn step1((x, depth): (i32, i32), cmd: &Command) -> (i32, i32) {
    match cmd {
        Command::Up(offs) => (x, depth - offs),
        Command::Down(offs) => (x, depth + offs),
        Command::Forward(offs) => (x + offs, depth),
    }
}

fn step2((x, depth, aim): (i32, i32, i32), cmd: &Command) -> (i32, i32, i32) {
    match cmd {
        Command::Up(offs) => (x, depth, aim - offs),
        Command::Down(offs) => (x, depth, aim + offs),
        Command::Forward(offs) => (x + offs, depth + aim * offs, aim),
    }
}

fn problem02() {
    let commands: Vec<Command> = read_lines(&data_file(2)).unwrap()
            .iter()
            .filter_map(|line| line.parse().ok())
            .collect();
    let pos1 = commands.iter().fold((0i32, 0i32), step1);
    let pos2 = commands.iter().fold((0i32, 0i32, 0i32), step2);
    println!("Answer 1: {}\nAnswer 2: {}\n", pos1.0 * pos1.1, pos2.0 * pos2.1);
}

problem02();

Answer 1: 1728414
Answer 2: 1765720035



In [5]:
fn get_most_common<'a>(nums: impl Iterator<Item = &'a Vec<u8>>, pos: usize) -> u8 {
    let mut n0 = 0;
    let mut n1 = 0;
    for num in nums {
        n0 += (num[pos] == 0) as usize;
        n1 += (num[pos] == 1) as usize;
    }
    if n1 >= n0 {
        1
    } else {
        0
    }
}

fn part1(nums: &Vec<Vec<u8>>) -> i64 {
    let mut lc: i64 = 0;
    let mut mc: i64 = 0;
    for i in 0..nums[0].len() {
        let m = get_most_common(nums.iter(), i) as i64;
        mc = mc * 2 + m;
        lc = lc * 2 + (1 - m);
    }
    mc * lc
}

fn digits_to_decimal(digits: &Vec<u8>)-> i64 {
    digits.iter().fold(0i64, |v, d| v * 2 + *d as i64)
}

fn part2(nums: &Vec<Vec<u8>>) -> i64 {
    let mut mnums: Vec<usize> = (0..nums.len()).collect();
    let mut lnums: Vec<usize> = (0..nums.len()).collect();
    for i in 0..nums[0].len() {
        if mnums.len() > 1 {
            let m = get_most_common(mnums.iter().cloned().map(|x| &nums[x]), i);
            mnums = mnums.iter().cloned().filter(|&x| nums[x][i] == m).collect();
        }
        if lnums.len() > 1 {
            let l = 1 - get_most_common(lnums.iter().cloned().map(|x| &nums[x]), i);
            lnums = lnums.iter().cloned().filter(|&x| nums[x][i] == l).collect();
        }
    }
    digits_to_decimal(&nums[mnums[0]]) * digits_to_decimal(&nums[lnums[0]])
}


fn problem03() {
    let nums: Vec<Vec<u8>> = read_lines(&data_file(3))
        .unwrap()
        .iter()
        .map(|line| line
            .chars()
            .map(|x| x.to_digit(10).unwrap() as u8)
            .collect())
        .collect();
    println!("Answer 1: {}\nAnswer 2: {}", part1(&nums), part2(&nums));
}

problem03();

Answer 1: 2972336
Answer 2: 3368358


In [62]:
type Board = Vec<Vec<i32>>;

fn is_full(board: &Board, pos: usize, is_vert: bool) -> bool {
    for i in 0..board.len() {
        let n = if is_vert {board[i][pos]} else {board[pos][i]};
        if n != -1 {
            return false;
        }
    }
    true
}

fn has_winner(board: &Board) -> bool {
    return (0..board.len())
        .any(|i| is_full(board, i, true) || is_full(board, i, false));
}

fn apply_num(board: &mut Board, num: i32) {
    for mut row in board {
        for i in 0..row.len() {
            if row[i] == num {
                row[i] = -1;
            }
        }
    }
}

fn run_simulation((nums, mut boards): (Vec<i32>, Vec<Board>)) -> Vec<i32> {
    let mut winners = vec![];
    let mut scores = vec![];
    for n in nums {
        for i in 0..boards.len() {
            if winners.contains(&i) {
                continue;
            }
            apply_num(&mut boards[i], n);
            if has_winner(&boards[i]) {
                let s: i32 = boards[i].iter().flatten().filter(|&x| *x != -1).sum();
                winners.push(i);
                scores.push(s * n);
            }
        }
    }
    scores
}

fn parse_game(lines: Vec<String>) -> (Vec<i32>, Vec<Board>) {
    let nums = lines[0].split(",").map(|s| s.parse::<i32>().unwrap()).collect();
    let mut boards = vec![];
    let mut board = vec![];
    for line in &lines[2..] {
        if line.is_empty() {
            boards.push(board);
            board = vec![];
        } else {
            board.push(line.split_whitespace()
                .map(|s| s.parse::<i32>().unwrap())
                .collect());
        }
    }
    boards.push(board);
    (nums, boards)
}

fn problem04() {
    let lines = read_lines(&data_file(4)).unwrap();
    let scores = run_simulation(parse_game(lines));
    println!("Answer 1: {}\nAnswer 2: {}", scores[0], scores.last().unwrap());
}

problem04();

Answer 1: 44736
Answer 2: 1827


In [124]:
extern crate num;
use num::signum;

type Point = (i32, i32);
type Line = (Point, Point);

fn parse_pt(pt: &str) -> Point {
    pt.split(",")
        .map(|x| x.parse::<i32>().unwrap())
        .tuple_windows()
        .next()
        .unwrap()
}

fn parse_line(line: &str) -> Line {
    line.split(" -> ")
        .map(parse_pt)
        .tuple_windows()
        .next()
        .unwrap()
}

fn stroke(
    (x1, y1): &Point, 
    (x2, y2): &Point, 
    ptmap: &mut HashMap<Point, u32>, 
    skip_diagonals: bool) {
    let dx = signum(x2 - x1);
    let dy = signum(y2 - y1);
    if skip_diagonals && dx != 0 && dy != 0 {
        return;
    }
    let mut cx = *x1;
    let mut cy = *y1;
    loop {
        *ptmap.entry((cx, cy)).or_insert(0) += 1;
        if cx == *x2 && cy == *y2 {
            break;
        }
        cx += dx;
        cy += dy;
    } 
}

fn find_num_overlaps(lines: &Vec<Line>, skip_diagonals: bool) -> usize {
    let mut ptmap: HashMap<Point, u32> = HashMap::new();
    for (a, b) in lines {
        stroke(a, b, &mut ptmap, skip_diagonals);
    }
    ptmap.values().filter(|&x| *x > 1).count()
}

fn problem05() {
    let lines: Vec<Line> = read_lines(&data_file(5))
        .unwrap()
        .iter()
        .map(|s| parse_line(&s)).collect();
    let res1 = find_num_overlaps(&lines, true);
    let res2 = find_num_overlaps(&lines, false);
    println!("Answer 1: {}\nAnswer 2: {}", res1, res2);
}

problem05();

Answer 1: 7436
Answer 2: 21104


In [53]:
fn get_n_spawned(n: u64, days: u64, spawned:&mut HashMap<(u64, u64), u64>) -> u64 {
    if days <= n {
        return 0;
    }
    let key = (n, days);
    if spawned.contains_key(&key) {
        return spawned[&key];
    }
    let mut res = (days - n - 1) /7 + 1;
    for i in 0..res {
        res += get_n_spawned(8, days - n - i * 7 - 1, spawned);
    }
    spawned.insert(key, res);
    res
}

fn sum_spawned(nums: &Vec<u64>, total_days: u64) -> u64 {
    let mut spawned: HashMap<(u64, u64), u64> = HashMap::new();
    let s: u64 = nums.iter().map(|x| get_n_spawned(*x, total_days, &mut spawned)).sum();
    s + nums.len() as u64
}

fn problem06() {
    let line: String = read_lines(&data_file(6)).unwrap().first().unwrap().clone();
    let nums: Vec<u64> = line.split(",").map(|x| x.parse::<u64>().unwrap()).collect();
    println!("Answer 1: {}\nAnswer 2: {}", sum_spawned(&nums, 80), sum_spawned(&nums, 256));
}

problem06();

Answer 1: 386755
Answer 2: 1732731810807


In [52]:

fn median(nums: &Vec<f64>) -> f64 {
    let mut nc = nums.clone();
    nc.sort_by(|a, b| a.partial_cmp(b).unwrap());
    let mid = nc.len() / 2;
    if nc.len() % 2 == 1 {
        nc[mid]
    } else {
        (nc[mid - 1] + nc[mid]) / 2.0
    }
}

fn part1(nums: &Vec<f64>) -> i32 {
    let minx = median(nums).round();
    let sum: f64 = nums.iter().map(|n| (n - minx).abs()).sum();
    sum as i32
}

fn min_gradient(p0: f64, f: impl Fn(f64) -> f64, num_it: usize) -> f64 {
    let EPS = 0.001;
    let mut p: f64 = p0;
    for i in 0..num_it {
        let pr = f(p + EPS);
        let pl = f(p - EPS);
        let dp = (pr - pl) / (2.0 * EPS);
        p = p - dp * EPS;
    }
    p
}

fn part2(nums: &Vec<f64>) -> i32 {
    let f = |x: f64| {
        let mut res = 0.0;
        for n in nums {
            let d = (x - n).abs();
            res += d * (d + 1.0) / 2.0;
        }
        res
    };
    let minx = min_gradient(median(&nums), f, 1000).round();
    f(minx) as i32
}

fn problem07() {
    let line: String = read_lines(&data_file(7)).unwrap().first().unwrap().clone();
    let nums: Vec<f64> = line.split(",").map(|x| x.parse::<f64>().unwrap()).collect();
    println!("Answer 1: {}\nAnswer 2: {}", part1(&nums), part2(&nums));
}

problem07();

Answer 1: 343468
Answer 2: 96086265


In [165]:
use std::str;
type Mapping = Vec<Vec<u8>>;

const CHARS: &str = "abcdefg";
const DIGITS: [&str; 10] = [
    "abcefg", "cf", "acdeg", "acdfg", "bcdf",
    "abdfg", "abdefg", "acf", "abcdefg", "abcdfg"];
const NUM_SEGMENTS: usize = 7;

fn normalize_mapping(mapping: &mut Mapping) {
    let mut changed = true;
    while changed {
        changed = false;
        for i in 0..mapping.len() {
            let c = &mapping[i];
            if c.iter().sum::<u8>() == 1 {
                let p = c.iter().position(|&e| e == 1).unwrap();
                for j in 0..NUM_SEGMENTS {
                    if j != i && mapping[j][p] == 1 {
                        mapping[j][p] = 0;
                        changed = true;
                    }
                }
            }
        }
    }
}

fn is_valid_mapping(mapping: &Mapping) -> bool {
    for i in 0..NUM_SEGMENTS {
        if mapping.iter().map(|m| m[i]).sum::<u8>() != 1 {
            return false;
        }
    }
    mapping.iter().map(|m| m.iter().sum::<u8>()).all(|s| s == 1)
}

fn get_mapping(mapping: &mut Mapping, combs: &Vec<&str>, pos: usize) -> Option<Mapping> {
    if pos == combs.len() {
        normalize_mapping(mapping);
        if is_valid_mapping(&mapping) {
            return Some(mapping.clone());
        } else {
            return None;
        }
    }
    
    let comb = &combs[pos];
    for digit in DIGITS {
        if digit.len() != comb.len() {
            continue;
        }
        let mut mapping1 = mapping.clone();
        for d in CHARS.bytes() {
            if !digit.as_bytes().contains(&d) {
                for &c in comb.as_bytes() {
                    mapping1[c as usize - 'a' as usize][d as usize - 'a' as usize] = 0;
                }
            }
        }            
        if let Some(res_mapping) = get_mapping(&mut mapping1, combs, pos + 1) {
            return Some(res_mapping);
        }
    }
    None
}

fn decode_digit(mapping: &Mapping, combs: &Vec<&str>) -> u64 {
    let mut res = 0;
    for comb in combs {
        let mut scomb = comb.as_bytes().to_vec();
        for i in 0..scomb.len() {
            scomb[i] = mapping[scomb[i] as usize - 'a' as usize]
                .iter()
                .position(|&e| e == 1)
                .map(|x| x + 'a' as usize)
                .unwrap() as u8;
        }
        scomb.sort();
        let s = str::from_utf8(&scomb).unwrap();
        let digit = DIGITS.iter().position(|&e| e == s).unwrap();
        res = res * 10 + digit;
    }
    res as u64
}

fn decode_sum(inp: &Vec<Vec<Vec<&str>>>) -> u64 {
    let mut res = 0;
    for line in inp {
        let combs = &line[0];
        let to_decode = &line[1];
        let mut mapping: Mapping = vec![vec![1; NUM_SEGMENTS]; NUM_SEGMENTS];
        let mut combs_sorted = combs.clone();
        combs_sorted.sort_by(|a, b| a.len().partial_cmp(&b.len()).unwrap()); 
        if let Some(res_mapping) = get_mapping(&mut mapping, &combs_sorted, 0) {
            res += decode_digit(&res_mapping, &to_decode);
        }
    }
    res as u64
}

fn problem08() {
    let lines = read_lines(&data_file(8)).unwrap();
    let inp: Vec<Vec<Vec<&str>>> = lines.iter()
        .map(|x| 
            x.split("|")
                .map(|s| s.trim().split(" ").collect())
                .collect())
        .collect();

    let res1: usize = inp.iter()
        .map(|v| v[1].iter().filter(|c| [2, 3, 4, 7].contains(&c.len())).count())
        .sum();
    let res2 = decode_sum(&inp);
    println!("Answer 1: {:?}\nAnswer 2: {:?}", res1, res2);
}

problem08();

Answer 1: 239
Answer 2: 946346


In [218]:
const OFFS: [(i32, i32); 4] = [(0, 1), (0, -1), (1, 0), (-1, 0)];

fn find_minima(board: &Vec<Vec<u8>>) -> Vec<(usize, usize)> {
    let mut res = vec![];
    let bh = board.len() as i32;
    for (j, row) in board.iter().enumerate() {
        for (i, h) in row.iter().enumerate() {
            let bw = row.len() as i32;
            let is_min = OFFS.iter()
                .all(|(dx, dy)| {
                    let x = i as i32 + dx;
                    let y = j as i32 + dy;
                    x < 0 || x >= bw || y < 0 || y >= bh || 
                        board[y as usize][x as usize] > *h
                });
            if is_min {
                res.push((i, j));
            }
        }
    }
    res
}

fn fill_cell(board: &Vec<Vec<u8>>, bidx: &mut Vec<Vec<i32>>, pos: (usize, usize), basin_idx: i32) {
    let (x, y) = pos;
    if x >= board[0].len() || y >= board.len() {
        return;
    }
    let b = board[y][x];
    if b == 9 {
        bidx[y][x] = 0;
        return;
    }
    if bidx[y][x] >= 0 {
        return;
    }
    bidx[y][x] = basin_idx;
    for (dx, dy) in OFFS {
        let (x1, y1) = ((x as i32 + dx) as usize, (y as i32 + dy) as usize);
        fill_cell(&board, bidx, (x1, y1), basin_idx);
    }
}


fn find_basins(board: &Vec<Vec<u8>>) -> Vec<Vec<i32>> {
    let w = board[0].len();
    let h = board.len();
    let mut bidx = vec![vec![-1; w]; h];
    let mut nbasins = 0;
    for j in 0..h {
        for i in 0..w {
            if bidx[j][i] == -1 {
                fill_cell(&board, &mut bidx, (i, j), nbasins + 1);
                nbasins += 1;
            }
        }
    }
    bidx
}

fn problem09() {
    let lines = read_lines(&data_file(9)).unwrap();
    let board: Vec<Vec<u8>> = lines.iter()
        .map(|x| x.as_bytes()
            .iter()
            .map(|c| c - '0' as u8)
            .collect())
        .collect();
    let mins = find_minima(&board);
    let res1: usize = mins.iter()
        .map(|(x, y)| board[*y][*x] as usize + 1)
        .sum();
    
    let basins = find_basins(&board);
    let max_basin = basins.iter().map(|x| x.iter().max().unwrap()).max().unwrap();
    
    let mut basin_sizes = vec![];
    let w = board[0].len();
    let h = board.len();
    for b in 1..max_basin + 1 {
        let mut size = 0;
        for j in 0..h {
            for i in 0..w {
                if basins[j][i] == b {
                    size += 1;
                }
            }
        }
        basin_sizes.push(size);
    }
    basin_sizes.sort();
    basin_sizes.reverse();
    let res2: usize = basin_sizes[0..3].iter().product();
    println!("Answer 1: {:?}\nAnswer 2: {:?}", res1, res2);
}

problem09();

Answer 1: 588
Answer 2: 964712


In [291]:
#[macro_use] extern crate lazy_static;
enum LineType {
    Corrupted(char),
    Incomplete(String)
}

fn problem10() {
    let lines = read_lines(&test_file(10, 1)).unwrap();
    let res1 = lines;
    let res2 = 0;
    println!("Answer 1: {:?}\nAnswer 2: {:?}", res1, res2);
}

problem10();

Answer 1: ["[({(<(())[]>[[{[]{<()<>>", "[(()[<>])]({[<{<<[]>>(", "{([(<{}[<>[]}>{[]{[(<()>", "(((({<>}<{<{<>}{[]{[]{}", "[[<[([]))<([[{}[[()]]]", "[{[{({}]{}}([{[{{{}}([]", "{<[[]]>}<{[{[{[]{()[[[]", "[<(<(<(<{}))><([]([]()", "<{([([[(<>()){}]>(<<{{", "<{([{{}}[<[[[<>{}]]]>[]]"]
Answer 2: 0


In [286]:
use std::collections::HashMap;
use std::iter::FromIterator;

type RuleSet = HashMap<String, String>;
type PatternMap = HashMap<String, usize>;

fn step(hist: &PatternMap, rules: &RuleSet) -> PatternMap {
    let mut res: PatternMap = HashMap::new();
    for (s, cnt) in hist {
        if rules.contains_key(s) {
            let c = &rules[s];
            *res.entry(s[0..1].to_string() + &c).or_insert(0) += cnt;
            *res.entry(c.to_string() + &s[1..2]).or_insert(0) += cnt;
        } else {
            res.insert(s.to_string(), *cnt);
        }
    }
    res
}

fn get_pairs_hist(pattern: &str) -> PatternMap {
    let mut res = HashMap::new();
    for i in 1..pattern.len() {
        *res.entry(pattern[i - 1..i + 1].to_string()).or_insert(0) += 1;
    }
    res
}

fn eval_pattern(pat: &str, rules: &RuleSet, steps: usize) -> usize {
    let mut hist = get_pairs_hist(&pat);
    for i in 0..steps {
        hist = step(&hist, &rules);
    }
        
    let mut char_hist = HashMap::new();
    for (s, cnt) in hist {
        *char_hist.entry(s[0..1].to_string()).or_insert(0) += cnt;
    }
    let last_char = pat[pat.len() - 1..pat.len()].to_string();
    *char_hist.entry(last_char).or_insert(0) += 1;
    
    let mut counts: Vec<&usize> = char_hist.values().collect();
    counts.sort();
    counts[counts.len() - 1] - counts[0]
}

fn problem14() {
    let lines = read_lines(&data_file(14)).unwrap();
    let start_pattern = lines[0].to_string();
    let rules: RuleSet = lines[2..].iter()
        .map(|s| { 
            let parts: Vec<&str> = s.split(" -> ").collect(); 
            (parts[0].to_string(), parts[1].to_string()) })
        .collect();
    let res1 = eval_pattern(&start_pattern, &rules, 10);
    let res2 = eval_pattern(&start_pattern, &rules, 40);
    println!("Answer 1: {:?}\nAnswer 2: {:?}", res1, res2);
}

problem14();

Answer 1: 2621
Answer 2: 2843834241366


In [319]:
use std::cmp::Ordering;
use std::collections::BinaryHeap;
use std::collections::HashMap;

#[derive(Copy, Clone, Eq, PartialEq)]
struct Node {
    score: usize,
    pos: (i32, i32),
}

impl Ord for Node {
    fn cmp(&self, other: &Self) -> Ordering {
        other.score.cmp(&self.score)
    }
}

impl PartialOrd for Node {
    fn partial_cmp(&self, other: &Self) -> Option<Ordering> {
        Some(self.cmp(other))
    }
}

const OFFS: [(i32, i32); 4] = [(0, 1), (0, -1), (1, 0), (-1, 0)];

fn get_risk(board: &Vec<Vec<u8>>, x: i32, y: i32) -> usize {
    let (w, h) = (board[0].len() as i32, board.len() as i32);
    let mut res = board[(y % h) as usize][(x % w) as usize] as usize;
    res += (x / w + y / h) as usize;
    if res > 9 {
        res = res % 10 + 1;
    }
    res
}

fn get_min_risk(board: &Vec<Vec<u8>>, pos: (i32, i32), goal: (i32, i32)) -> usize {
    let (w, h) = (board[0].len() as i32, board.len() as i32);
    
    let mut to_explore = BinaryHeap::new();
    to_explore.push(Node { score: 0, pos: pos });
    
    let mut scores = HashMap::new();
    scores.insert(pos, 0usize);
    
    while let Some(Node { score, pos }) = to_explore.pop() {
        let (x, y) = pos;
        if pos == goal {
            return scores[&pos];
        }
        for (dx, dy) in OFFS {
            let (cx, cy) = (x + dx, y + dy);
            if cx < 0 || cy < 0 || cx > goal.0 || cy > goal.1 {
                continue;
            }
            let cpos = (cx, cy);
            let cscore = score + get_risk(&board, cx, cy);
            if !scores.contains_key(&cpos) || scores[&cpos] > cscore {
                scores.insert(cpos, cscore);
                to_explore.push(Node { score: cscore, pos: cpos });
            }
        }
    }
    0
}

fn problem15() {
    let lines = read_lines(&data_file(15)).unwrap();
    let board: Vec<Vec<u8>> = lines.iter()
        .map(|x| x.as_bytes()
            .iter()
            .map(|c| c - '0' as u8)
            .collect())
        .collect();
    let (w, h) = (board[0].len() as i32, board.len() as i32);
    let res1 = get_min_risk(&board, (0, 0), (w - 1, h - 1));
    let res2 = get_min_risk(&board, (0, 0), (w * 5 - 1, h * 5 - 1));
    println!("Answer 1: {:?}\nAnswer 2: {:?}", res1, res2);
}

problem15();

Answer 1: 527
Answer 2: 2887


In [530]:
use std::fmt;
use std::ptr;
use std::iter::Peekable;

#[derive(PartialEq)]
enum ReduceOp {
    None,
    Explode,
    Split,
}

#[derive(Clone)]
enum SFNum {
    Regular(u32),
    Nested((Box<SFNum>, Box<SFNum>)),
}

impl fmt::Debug for SFNum {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            SFNum::Regular(n) => write!(f, "{}", n),
            SFNum::Nested((a, b)) => write!(f, "[{:?},{:?}]", a, b),
        }
    }
}

impl SFNum {
    fn magnitude(&self) -> u32 {
        match self {
            SFNum::Regular(n) => *n,
            SFNum::Nested((a, b)) => 3 * a.magnitude() + 2 * b.magnitude(),
        }
    }
    
    fn add(&self, other: &SFNum) -> SFNum {
        SFNum::Nested((Box::new(self.clone()), Box::new(other.clone())))
    }
}

fn parse_sf_num<T: Iterator<Item = char>>(it: &mut Peekable<T>) -> Result<SFNum, String> {
    while let Some(&c) = it.peek() {
        match c {
            _ if char::is_digit(c, 10) => {
                let mut num = 0;
                while let Some(&n) = it.peek() {
                    if !char::is_digit(n, 10) {
                        break;
                    }
                    num = num * 10 + (n as u32 - '0' as u32);
                    it.next();
                }
                return Ok(SFNum::Regular(num));
            },
            '[' => {
                it.next();
                let a = parse_sf_num(it)?;
                let b = parse_sf_num(it)?;
                if it.peek() != Some(&']') {
                    return Err("Expecting closing ']'".to_string());
                }
                it.next();
                return Ok(SFNum::Nested((Box::new(a), Box::new(b))));
            },
            ',' => { it.next(); },
            _ if char::is_whitespace(c) => { it.next(); },
            _ => return Err(format!("Invalid character: '{}'", c)),
        }
    }
    Err("Incomplete stream".to_string())
}

fn reduce_num(num: &mut SFNum, can_split: bool) -> ReduceOp {
    struct Context {
        prev_regular: *mut SFNum,
        next_regular_inc: u32,
        current_op: ReduceOp,
        can_split: bool,
    }
    
    fn add_to_prev_regular(val: u32, ctx: &mut Context) {
        if !ctx.prev_regular.is_null() {
            unsafe {
                match *ctx.prev_regular {
                    SFNum::Regular(prevn) => 
                        *ctx.prev_regular = SFNum::Regular(val + prevn),
                    _ => {}
                }
            }
        }
    }
    
    fn rec(num: &mut SFNum, depth: u32, ctx: &mut Context) {
        if ctx.current_op != ReduceOp::None {
            match num {
                SFNum::Regular(n) => {
                    *num = SFNum::Regular(*n + ctx.next_regular_inc);
                    ctx.next_regular_inc = 0;
                }
                SFNum::Nested((a, b)) => {
                    rec(a, depth + 1, ctx);
                    rec(b, depth + 1, ctx);
                }
            }
            return;
        }
        match num {
            SFNum::Nested((a, b)) => {
                match (&mut **a, &mut **b) {
                    (SFNum::Regular(n0), SFNum::Regular(n1)) if depth >= 4 => {
                        add_to_prev_regular(*n0, ctx);
                        ctx.next_regular_inc = *n1;
                        *num = SFNum::Regular(0);
                        ctx.current_op = ReduceOp::Explode;
                        return;
                    },
                    _ => {
                        rec(a, depth + 1, ctx);
                        rec(b, depth + 1, ctx);
                    }
                }
            },
            SFNum::Regular(n) => {
                if ctx.can_split && *n >= 10 {
                    if depth >= 4 {
                        add_to_prev_regular(*n / 2, ctx);
                        ctx.next_regular_inc = (*n + 1) / 2;
                        *num = SFNum::Regular(0);
                        ctx.current_op = ReduceOp::Explode;
                        return;
                    } else {
                        *num = SFNum::Nested((
                            Box::new(SFNum::Regular(*n / 2)), 
                            Box::new(SFNum::Regular((*n + 1) / 2))
                        ));
                        ctx.current_op = ReduceOp::Split;
                        return;
                    }
                }
                ctx.prev_regular = num;
            },
        }
    }
    let mut ctx: Context = Context { 
        prev_regular: ptr::null_mut(), 
        next_regular_inc: 0, 
        current_op: ReduceOp::None,
        can_split: can_split,
    };
    rec(num, 0, &mut ctx);
    ctx.current_op
}

fn eval_nums(nums: &Vec<SFNum>) -> SFNum {
    let mut res = nums[0].clone();
    for i in 1..nums.len() {
        res = res.add(&nums[i]);
        loop {
            let op1 = reduce_num(&mut res, false);
            if op1 == ReduceOp::Explode {
                continue;
            }
            let op2 = reduce_num(&mut res, true);
            if op2 == ReduceOp::None {
                break;
            }
        }
    }
    res
}

fn problem18() {
    let lines = read_lines(&data_file(18)).unwrap();
    let sfnums: Vec<SFNum> = lines.iter()
        .map(|s| parse_sf_num(&mut s.chars().peekable()).unwrap())
        .collect();
    let res1 = eval_nums(&sfnums).magnitude();
    
    let mut res2 = 0;
    for i in 0..sfnums.len() {
        for j in 0..sfnums.len() {
            if i != j {
                let num = eval_nums(&vec![sfnums[i].clone(), sfnums[j].clone()]);
                res2 = res2.max(num.magnitude());
            }
        }
    }
    
    println!("Answer 1: {:?}\nAnswer 2: {:?}", res1, res2);
}

problem18();

Answer 1: 3725
Answer 2: 4832


In [27]:
extern crate itertools;
use itertools::Itertools;

type DirMap = HashMap<Vec3, (usize, usize)>;

#[derive(PartialEq, Eq, Hash, Clone, Debug)] 
struct Vec3 {
    pub x: i32,
    pub y: i32,
    pub z: i32,
}

#[derive(Debug)]
struct Mat3 {
    pub row: [Vec3; 3],
}

impl Vec3 {
    fn dot(&self, other: &Vec3) -> i32 {
        self.x * other.x + self.y * other.y + self.z * other.z
    }
    
    fn cross(&self, other: &Vec3) -> Vec3 {
        Vec3{
            x: -other.y * self.z + self.y * other.z, 
            y: other.x * self.z - self.x * other.z, 
            z: -other.x * self.y + self.x * other.y,
        }
    }
    
    fn add(&self, other: &Vec3) -> Vec3 {
        Vec3 {
            x: self.x + other.x,
            y: self.y + other.y,
            z: self.z + other.z,
        }
    }
    
    fn sub(&self, other: &Vec3) -> Vec3 {
        Vec3 {
            x: self.x - other.x,
            y: self.y - other.y,
            z: self.z - other.z,
        }
    }
    
    fn transform(&self, m: &Mat3) -> Vec3 {
        Vec3 {
            x: self.dot(&m.row[0]),
            y: self.dot(&m.row[1]),
            z: self.dot(&m.row[2]),
        }
    }
    
    fn set(&mut self, pos: usize, val: i32) {
        match pos {
            0 => self.x = val,
            1 => self.y = val,
            2 => self.z = val,
            _ => panic!("Invalid Vec3 component index"),
        }
    }
    
    fn abs(&self) -> Vec3 {
        Vec3 {x: self.x.abs(), y: self.y.abs(), z: self.z.abs()}
    }
    
    fn reverse(&self) -> Vec3 {
        Vec3 {x: -self.x, y: -self.y, z: -self.z}
    }
    
    fn zero() -> Vec3 {
        Vec3 {x: 0, y: 0, z: 0}
    }
    
    fn manhattan_dist(&self, other: &Vec3) -> i32 {
        (self.x - other.x).abs() + (self.y - other.y).abs() + (self.z - other.z).abs()
    }
}

impl Mat3 {
    fn zero() -> Mat3 {
        Mat3 {row: [Vec3::zero(), Vec3::zero(), Vec3::zero()] }
    }
}

fn enum_transforms() -> Vec<Mat3> {
    let mut res = vec![];
    for i in 0..3 {
        for sign1 in [1, -1] {
            for j in 0..3 {
                if i == j {
                    continue;
                }
                for sign2 in [1, -1] {
                    let mut tm = Mat3::zero();
                    tm.row[0].set(i, sign1);
                    tm.row[1].set(j, sign2);
                    tm.row[2] = tm.row[0].cross(&tm.row[1]);
                    res.push(tm);
                }
            }
        }
    }
    res
}

fn compute_dirs(coords: &Vec<Vec3>) -> HashMap<Vec3, (usize, usize)> {
    let mut res = HashMap::new();
    for i in 0..coords.len() {
        for j in 0..coords.len() {
            res.insert(coords[j].sub(&coords[i]).abs(), (i, j));
        }
    }
    res
}

fn get_offset(coords1: &Vec<Vec3>, coords2: &Vec<Vec3>, dirs1: &DirMap, dirs2: &DirMap) -> Option<Vec3> {
    let mut num_matching_dirs = 0;
    let mut candidate_point1 = 0;
    let mut candidate_points2 = vec![];
    for v in dirs1.keys() {
        if dirs2.contains_key(&v) {
            num_matching_dirs += 1;
            if candidate_points2.len() == 0 {
                let (a, b) = dirs2[&v];
                candidate_points2.push(a);
                candidate_points2.push(b);
                candidate_point1 = dirs1[&v].0;
            }
        }
    }
    if num_matching_dirs < 66 {
        return None;
    }
    
    let coords2_set: HashSet<&Vec3> = coords2.iter().collect();
    for cand_pt in candidate_points2 {
        let c1 = &coords1[candidate_point1];
        let c2 = &coords2[cand_pt];
        let offs = c2.sub(&c1);
        let mut num_matching_pos = 0;
        for v in coords1 {
            if coords2_set.contains(&v.add(&offs)) {
                num_matching_pos += 1;
            }
        }
        if num_matching_pos >= 12 {
            return Some(offs);
        }
    }
    None
}

fn get_scanner_positions(data: &Vec<Vec<Vec3>>) -> Option<(Vec<Vec3>, Vec<Vec<Vec3>>)> {
    struct Context<'a> {
        res: Vec<Vec3>,
        found: HashSet<usize>,
        to_find: HashSet<usize>,
        non_overlapping: HashSet<(usize, usize)>,
        dirs: HashMap<(usize, usize), DirMap>,
        coords: HashMap<(usize, usize), Vec<Vec3>>,
        data: &'a Vec<Vec<Vec3>>,
        transforms: Vec<Mat3>,
    }
    
    let mut ctx = Context {
        res: vec![Vec3::zero(); data.len()],  
        found: vec![0].into_iter().collect(),
        to_find: (1..data.len()).into_iter().collect(),
        non_overlapping: HashSet::new(),
        dirs: HashMap::new(),
        coords: HashMap::new(),
        data: data,
        transforms: enum_transforms(),
    };
    
    // precompute both transformed coordinates and directions
    for i in 0..data.len() {
        for (k, tm) in ctx.transforms.iter().enumerate() {
            let coords_tm = data[i].iter().map(|p| p.transform(&tm)).collect();
            ctx.dirs.insert((i, k), compute_dirs(&coords_tm));
            ctx.coords.insert((i, k), coords_tm);
        }
    }
           
    fn test_pair(i: usize, j: usize, ctx: &mut Context) -> bool {
        if ctx.non_overlapping.contains(&(i, j)) {
            return false;
        }
        for (k, tm) in ctx.transforms.iter().enumerate() {
            match get_offset(
                &ctx.coords[&(j, 0)],
                &ctx.coords[&(i, k)], 
                &ctx.dirs[&(j, 0)], 
                &ctx.dirs[&(i, k)]) {
                None => continue,
                Some(offs) => {
                    ctx.coords.insert((i, 0), 
                        ctx.coords[&(i, k)].iter()
                            .map(|pt| pt.sub(&offs))
                            .collect());
                    ctx.dirs.insert((i, 0), compute_dirs(&ctx.coords[&(i, k)]));
                    ctx.found.insert(i);
                    ctx.to_find.remove(&i);
                    ctx.res[i] = offs.reverse();
                    return true;
                }
            }
        }
        ctx.non_overlapping.insert((i, j));
        false
    }
    
    while ctx.to_find.len() > 0 {
        let pairs: Vec<(usize, usize)> = ctx.to_find.iter()
            .cartesian_product(ctx.found.iter())
            .map(|(i, j)| (*i, *j))
            .collect();
        let mut found = false;
        for (i, j) in pairs {
            if test_pair(i, j, &mut ctx) {
                found = true;
                break;
            }
        }
        if !found {
            println!("No solution");
            return None;
        }
    }
    let data_tm = (0..data.len()).map(|i| ctx.coords[&(i, 0)].clone()).collect();
    let res = ctx.res;
    Some((res, data_tm))
}

fn problem19() {
    let lines = read_lines(&data_file(19)).unwrap();
    let mut data: Vec<Vec<Vec3>> = lines.split(|s| s.len() == 0)
        .map(|lines| 
            lines[1..].iter().map(|line| {
                let mut parts = line.split(",").map(|n| n.parse::<i32>().unwrap());
                Vec3 {x: parts.next().unwrap(), y: parts.next().unwrap(), z: parts.next().unwrap()}
            }).collect())
    .collect();
    
    if let Some((scanner_pos, data_tm)) = get_scanner_positions(&mut data) {
        let mut bpos: HashSet<Vec3> = HashSet::new();
        for d in data_tm {
            for p in d {
                bpos.insert(p);
            }
        }
        let res1 = bpos.len();

        let mut res2 = 0;
        for i in 0..scanner_pos.len() {
            for j in i + 1..scanner_pos.len() {
                res2 = res2.max(scanner_pos[i].manhattan_dist(&scanner_pos[j]));
            }
        }
        println!("Answer 1: {:?}\nAnswer 2: {:?}", res1, res2);
    }
}

problem19();

Answer 1: 385
Answer 2: 10707
